In [ ]:
import warnings
warnings.filterwarnings("ignore")
import random
from PIL import Image
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import cv2
import glob as glob
from tqdm.auto import tqdm
import random

from sklearn.model_selection import train_test_split

import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras.models import Model
from tensorflow.keras import layers 
from tensorflow.keras.layers import Conv2D , MaxPooling2D , BatchNormalization , Conv2DTranspose ,UpSampling2D
from tensorflow.keras.callbacks import ModelCheckpoint,  ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import backend as K


In [ ]:
AUTOTUNE=tf.data.experimental.AUTOTUNE

In [ ]:
sns.set_theme(style='darkgrid', palette='pastel')
color = sns.color_palette(palette='pastel')

In [ ]:
IMG_DIR="/kaggle/input/ham1000-segmentation-and-classification/images"
MASK_DIR="/kaggle/input/ham1000-segmentation-and-classification/masks"
IMG_SIZE=256
BATCH_SIZE=32
BUFFER_SIZE=100

In [ ]:
def img_and_mask_path(img_dir,mask_dir):
    img_path=sorted(glob.glob(os.path.join(img_dir,"*.jpg")))
    masks_path=sorted(glob.glob(os.path.join(mask_dir,"*.png")))
    
    img_path=np.array(img_path)
    print(img_path)
    mask_path=np.array(masks_path)
    return img_path,mask_path

In [ ]:
def create_dir(path):
    os.makedirs(path, exist_ok=True)

In [ ]:
img_path,mask_path=img_and_mask_path(IMG_DIR,MASK_DIR)

In [ ]:
create_dir("files")

In [ ]:
def load_image_and_mask(img_path,mask_path):
    img=Image.open(img_path).convert("RGB")
    mask=Image.open(mask_path).convert("L")
    return np.array(img),np.array(mask)
    
random_indices=random.sample(range(len(img_path)),4)

images=[load_image_and_mask(img_path[i],mask_path[i])[0] for i in random_indices]
masks=[load_image_and_mask(img_path[i],mask_path[i])[1] for i in random_indices]  
fig,ax=plt.subplots(4,2,figsize=(10,20))
for i in range(4):
    ax[i,0].imshow(images[i])
    ax[i,0].set_title(f"Image {random_indices[i]}")
    ax[i, 0].axis('off')

    ax[i,1].imshow(masks[i],cmap="gray")
    ax[i,1].set_title(f"Mask {random_indices[i]}")
    ax[i,1].axis('off')

plt.tight_layout()
plt.savefig("files/Skin_Cancer_Mask_Image.jpg")
plt.show()

In [ ]:
x_train , x_test , y_train , y_test = train_test_split(img_path , mask_path , train_size = 0.80)

In [ ]:
print(f"Length of X_train Dataset {len(x_train)}")
print(f"Length of X_test Dataset {len(x_test)}")
print(f"Length of y_train Dataset {len(y_train)}")
print(f"Length of y_test Dataset {len(y_test)}")

In [ ]:
x_train , x_val , y_train , y_val = train_test_split(x_train , y_train , train_size = 0.80)

In [ ]:
print(f"Length of X_train Dataset {len(x_train)}")
print(f"Length of X_val Dataset {len(x_val)}")
print(f"Length of y_train Dataset {len(y_train)}")
print(f"Length of y_val Dataset {len(y_val)}")

In [ ]:
smooth = 1e-15

def dice_coef(y_true,y_pred):
    y_true_flat=tf.keras.layers.Flatten()(y_true)
    y_pred_flat=tf.keras.layers.Flatten()(y_pred)
    intersection=tf.reduce_sum(y_true*y_pred)
    return (2.*intersection+smooth)/(tf.reduce_sum(y_true)+tf.reduce_sum(y_pred)+smooth)

def dice_loss(y_true,y_pred):
    return 1.0-dice_coef(y_true,y_pred)

def iou(y_true, y_pred, smooth=1e-5):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    
    intersection = K.sum(y_true * y_pred, axis=[1, 2, 3])
    union = K.sum(y_true, axis=[1, 2, 3]) + K.sum(y_pred, axis=[1, 2, 3]) - intersection
    
    iou = (intersection + smooth) / (union + smooth)
    
    return K.mean(iou)

In [ ]:
def map_fn(img_path,mask_path):
    img=tf.io.read_file(img_path)
    img=tf.image.decode_jpeg(img,channels=3)
    img=tf.image.resize(img,(IMG_SIZE,IMG_SIZE))
    img=tf.cast(img,tf.float32)/255.0
    
    mask=tf.io.read_file(mask_path)
    mask=tf.image.decode_png(mask,channels=1)
    mask=tf.image.resize(mask,(IMG_SIZE,IMG_SIZE))
    mask=tf.cast(mask,tf.float32)/255.0
    return img,mask

In [ ]:
train_set=tf.data.Dataset.from_tensor_slices((x_train,y_train))
train_set=train_set.map(map_fn,num_parallel_calls=AUTOTUNE)
train_set=train_set.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

val_set=tf.data.Dataset.from_tensor_slices((x_val,y_val))
val_set=val_set.map(map_fn,num_parallel_calls=AUTOTUNE)
val_set=val_set.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

test_set=tf.data.Dataset.from_tensor_slices((x_test,y_test))
test_set=test_set.map(map_fn,num_parallel_calls=AUTOTUNE)
test_set=test_set.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)

In [ ]:
print(f"the size of the train dataloader {len(train_set)} batches of {BATCH_SIZE}")
print(f"the size of the test dataloader {len(test_set)} batches of {BATCH_SIZE}")
print(f"the size of the val dataloader {len(val_set)} batches of {BATCH_SIZE}")

In [ ]:
img_sample , mask_sample = next(iter(train_set)) 
img_sample.shape , mask_sample.shape

In [ ]:
fig , axis = plt.subplots(3 , 2 , figsize = (15,10)) 
for i in range(3): 
    img1 = img_sample[i].numpy()
    axis[i, 0].imshow(img1)
    axis[i, 0].set(title = f"Original Image")
    axis[i, 0].axis('off')
    
    
    img2 = mask_sample[i].numpy()
    axis[i, 1].imshow(img2)
    axis[i, 1].set(title = f"Mask Image")
    axis[i, 1].axis('off')

plt.subplots_adjust(wspace=0.0)
plt.savefig("files/skin_cancer_mask_and_image.jpg")

In [ ]:
def single_conv_block(filters, input_tensor):
    x = layers.Conv2D(filters, 3, strides=1, padding="same")(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    return x

def double_conv_block(filters, input_tensor):
    x = single_conv_block(filters, input_tensor)
    x = single_conv_block(filters, x)
    return x


In [ ]:
def UNET() : 
    inputs = keras.Input(shape = (IMG_SIZE , IMG_SIZE , 3))
    
    x = Conv2D(32, 3, strides=1, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    
    skip_connections = [] 
    
    for filters in [64, 128, 256 , 512]:
        x = Conv2D(filters , 3 , strides = 1 , padding = 'same')(x)
        x = BatchNormalization()(x)
        x = layers.Activation("relu")(x)

        x = Conv2D(filters , 3 , strides = 1 , padding = 'same')(x)
        x = BatchNormalization()(x)
        x = layers.Activation("relu")(x)
        
        skip_connections.append(x)

        x = MaxPooling2D(2, strides=2, padding="same")(x)
        
    x = Conv2D(1024 , 3 , strides = 1 , padding = 'same')(x)
    x = BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = Conv2D(1024 , 3 , strides = 1 , padding = 'same')(x)
    x = BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    
    for filters in [512, 256 , 128 , 64] : 
        x = Conv2DTranspose(filters , 2 , strides = 2 , padding = "same")(x)
        skip_connection = skip_connections.pop()
        x = layers.add([x , skip_connection])
        
        x = Conv2D(filters , 3 , strides = 1 , padding = 'same')(x)
        x = BatchNormalization()(x)
        x = layers.Activation("relu")(x)

        x = Conv2D(filters , 3 , strides = 1 , padding = 'same')(x)
        x = BatchNormalization()(x)
        x = layers.Activation("relu")(x)
        
    outputs = Conv2D(1 , 1 , strides = 1 , activation = "sigmoid")(x)
        
    
    model = Model(inputs , outputs)
    
    return model

In [ ]:
model = UNET()
model.summary()

In [ ]:
keras.utils.plot_model(model, show_shapes=True)


In [ ]:
EPOCHS=12

In [ ]:
learning_tate=1e-4
model.compile(tf.keras.optimizers.Adam(learning_tate) ,
              loss=dice_loss,
              metrics=['accuracy', dice_coef, iou])

In [ ]:
model_path="/kaggle/working/files/skin_cancer_segmentation.keras"
csv_path="/kaggle/working/files/logger.csv"

In [ ]:
callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-6, verbose=1),
        EarlyStopping(monitor='val_loss', patience=25, restore_best_weights=False)
    ]


In [ ]:
history = model.fit(
        train_set , 
        epochs = EPOCHS , 
        batch_size = BATCH_SIZE , 
        validation_data = val_set , 
        callbacks=callbacks
    )


In [ ]:
model.save("skin_cancer_segmentation.keras")

In [ ]:
history.history.keys()

In [ ]:
round(history.history["accuracy"][-1]*100,2)

In [ ]:
print(f"The Training accuracy is: {round(history.history['accuracy'][-1]*100, 2)}")
print(f"The Training dice coef is: {round(history.history['dice_coef'][-1]*100, 2)}")
print(f"The Training iou is: {round(history.history['iou'][-1]*100, 2)}")
print(f"The Training loss is: {round(history.history['loss'][-1]*100, 2)}")
print(f"The Training val accuracy is: {round(history.history['val_accuracy'][-1]*100, 2)}")
print(f"The Training val dice coef is: {round(history.history['val_dice_coef'][-1]*100, 2)}")
print(f"The Training val iou is: {round(history.history['val_iou'][-1]*100, 2)}")
print(f"The Training val loss is: {round(history.history['val_loss'][-1]*100, 2)}")
print(f"The learning rate is: {round(history.history['learning_rate'][-1]*100, 2)}")


In [ ]:
fig,axis=plt.subplots(2,2,figsize=(15,15))

axis[0][0].plot(history.history["loss"],label="train")
axis[0][0].plot(history.history["val_loss"],label="test")
axis[0][0].set_title("Training/Testing loss over epochs")
axis[0][0].set_xlabel("Epochs")
axis[0,0].set_ylabel('loss')
axis[0,0].legend()


axis[1,0].plot(history.history['accuracy'], label='train')
axis[1,0].plot(history.history['val_accuracy'], label='test')
axis[1,0].set_title('Training/Testing accuracy over Epochs')
axis[1,0].set_xlabel('epoch')
axis[1,0].set_ylabel('Accuracy')
axis[1,0].legend()


axis[0,1].plot(history.history['iou'], label='train')
axis[0,1].plot(history.history['val_iou'], label='test')
axis[0,1].set_title('Training/Testing accuracy over Epochs')
axis[0,1].set_xlabel('epoch')
axis[0,1].set_ylabel('iou')
axis[0,1].legend()



axis[1,1].plot(history.history['dice_coef'], label='train')
axis[1,1].plot(history.history['val_dice_coef'], label='test')
axis[1,1].set_title('Training/Testing dice coef over Epochs')
axis[1,1].set_xlabel('epoch')
axis[1,1].set_ylabel('dice coef')
axis[1,1].legend()

plt.savefig("files/skin_cancer_results.jpg")
plt.show()


In [ ]:
x_testsample,y_testsample=next(iter(test_set))

In [ ]:
def draw_images(test_images,test_masks,y_preds):
    plt.figure(figsize=(20,20))
    index=0
    n=np.random.randint(y_preds.shape[0])
    for i in range(18):
        plt.subplot(3,6,(i+1))
        if index==0:
            plt.imshow(test_images[n])
            plt.imshow(test_images[n])
            plt.title("Original Image")
            index=1
            plt.axis("off")
        
        elif index==1:
            plt.imshow(test_masks[n])
            plt.imshow(test_masks[n],alpha=0.2,cmap="jet")
            plt.title("Original Mask")
            index=2
            plt.axis("off")
            
        elif index==2:
            plt.imshow(test_masks[n])
            plt.imshow(y_preds[n],alpha=0.2,cmap="jet")
            plt.title("Predict Mask")
            index=0
            n=np.random.randint(y_preds.shape[0])
            plt.axis("off")
            
    plt.savefig("files/skin_cancer_segmented_mask_image.jpg")
    plt.legend()
    plt.show()